<a href="https://colab.research.google.com/github/papipopapu/PhysicsThings/blob/main/FluidSym.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import numba as nb
import typing as tp

In [2]:
from scipy.interpolate import RectBivariateSpline 

In [ ]:
x = np.arange(10)
y = np.arange(10)
z = np.arange(100)
z = z.reshape((10,10))

In [ ]:
mySpline = RectBivariateSpline(x, y, z)

In [ ]:
h = mySpline.ev([0,0], [1,4.5])
h

array([1. , 4.5])

In [ ]:
z

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
       [30, 31, 32, 33, 34, 35, 36, 37, 38, 39],
       [40, 41, 42, 43, 44, 45, 46, 47, 48, 49],
       [50, 51, 52, 53, 54, 55, 56, 57, 58, 59],
       [60, 61, 62, 63, 64, 65, 66, 67, 68, 69],
       [70, 71, 72, 73, 74, 75, 76, 77, 78, 79],
       [80, 81, 82, 83, 84, 85, 86, 87, 88, 89],
       [90, 91, 92, 93, 94, 95, 96, 97, 98, 99]])

In [ ]:
for i in range(100):
  m = RectBivariateSpline(x, y, np.random.rand(10,10))

In [66]:



class fluidThings:
  def __init__(self, N: int, dh: float):
    self.__N = N # Number of fluid cells on a row or column (for now we have a square)
    self.__dh = dh # Size of a cell 

    longT = np.arange(N+1) # temp things
    shortT = np.arange(N)

    self.__u = np.arange(N*(N+1)).reshape(N, N+1) # Field of vx velocities in walls (named u)
    self.__uSpline = RectBivariateSpline(shortT, longT, self.__u) # Smooth interpolation
    print (self.__u ) # to remove check

 
    self.__v = np.arange(N*(N+1)).reshape(N+1, N) # Field of vy velocities in walls (named v)
    self.__vSpline = RectBivariateSpline(longT, shortT, self.__v)
    print (self.__v )

  @staticmethod
  @nb.njit()
  def rToUij(r: np.array, N: int, dh: float): # ret = [[ui],[uj],[vi],[vj]], this functions gets x,y and returns indices for u and v in np.array
    ret = np.zeros((4, r[0].size));
    xScaled = r[0]/dh # temp useful things
    yScaled = r[1]/dh

    ret[0] = -yScaled + N - 0.5 # u_i
    ret[1] = xScaled # u_j
    ret[2] = -yScaled + N # v_i
    ret[3] = xScaled - 0.5 # v_j

    return ret


  def fG(self, r: np.array): # r = [[x],[y]]

    idx = self.rToUij(r, self.__N, self.__dh); # get indices
    uVector = np.vstack((self.__uSpline.ev(idx[0], idx[1]), self.__vSpline.ev(idx[2], idx[3]))) # interpolate whole vector

    return uVector





In [67]:
fluid = fluidThings(10, 1)

[[  0   1   2   3   4   5   6   7   8   9  10]
 [ 11  12  13  14  15  16  17  18  19  20  21]
 [ 22  23  24  25  26  27  28  29  30  31  32]
 [ 33  34  35  36  37  38  39  40  41  42  43]
 [ 44  45  46  47  48  49  50  51  52  53  54]
 [ 55  56  57  58  59  60  61  62  63  64  65]
 [ 66  67  68  69  70  71  72  73  74  75  76]
 [ 77  78  79  80  81  82  83  84  85  86  87]
 [ 88  89  90  91  92  93  94  95  96  97  98]
 [ 99 100 101 102 103 104 105 106 107 108 109]]
[[  0   1   2   3   4   5   6   7   8   9]
 [ 10  11  12  13  14  15  16  17  18  19]
 [ 20  21  22  23  24  25  26  27  28  29]
 [ 30  31  32  33  34  35  36  37  38  39]
 [ 40  41  42  43  44  45  46  47  48  49]
 [ 50  51  52  53  54  55  56  57  58  59]
 [ 60  61  62  63  64  65  66  67  68  69]
 [ 70  71  72  73  74  75  76  77  78  79]
 [ 80  81  82  83  84  85  86  87  88  89]
 [ 90  91  92  93  94  95  96  97  98  99]
 [100 101 102 103 104 105 106 107 108 109]]


In [70]:
r = np.array([[9.5],[10]]) 
u = fluid.fG(r)

ret:
[[-0.5]
 [ 9.5]
 [ 0. ]
 [ 9. ]]


In [71]:
u

array([[9.5],
       [9. ]])